In [1]:
# Part 1 : gensim LDA based on NLTK & SpaCy

# Run in python console
import nltk; nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ipekcinar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# For Shakespeare : We need to change this: kill thou, thy and shall & keep subject, re, edu and use
stop_words.extend(['may', 'make', 'would', 'shall', 'must', 'could'])  # make, come, go also very common


In [4]:
# Import Dataset -- original source
'''
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])
'''

'\ndf = pd.read_json(\'https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json\')\nprint(df.target_names.unique())\ndf.head()\n\n# Convert to list\ndata = df.content.values.tolist()\n\n# Remove Emails\ndata = [re.sub(\'\\S*@\\S*\\s?\', \'\', sent) for sent in data]\n\n# Remove new line characters\ndata = [re.sub(\'\\s+\', \' \', sent) for sent in data]\n\n# Remove distracting single quotes\ndata = [re.sub("\'", "", sent) for sent in data]\n\npprint(data[:1])\n'

In [5]:
# Import Dataset -- Shakespeare

# Pull all into "data" = all Shakespeare raw text

# directory containing all source texts for training the model 
data_dir="/Users/ipekcinar/Desktop/populism-hackathon/corpus_populism/United States/USPresidentialSpeeches"
import glob, os
os.chdir(data_dir)

#documents = list()
data = list()   # reset data to 0

for filename in glob.glob("*.txt"):
    filedata = open(filename, 'r').read()
    print(filename + " = " + str(len(filedata)) + " chars")
    #documents = documents + filedata.split(".")
    data.append(filedata)


1970-04-30 Richard Milhous Nixon.txt = 15741 chars
1864-12-06 Abraham Lincoln.txt = 32761 chars
1998-08-17 Bill Clinton.txt = 2986 chars
1978-01-19 Jimmy Carter.txt = 26275 chars
1941-03-15 Franklin Delano Roosevelt.txt = 16836 chars
1871-12-04 Ulysses S. Grant.txt = 32761 chars
2008-01-28 George W. Bush.txt = 32761 chars
1790-12-29 George Washington.txt = 7789 chars
1961-05-25 John Fitzgerald Kennedy.txt = 32761 chars
1966-06-30 Lyndon Baines Johnson.txt = 21181 chars
1961-02-21 John Fitzgerald Kennedy.txt = 5338 chars
1811-11-05 James Madison.txt = 13747 chars
1977-05-22 Jimmy Carter.txt = 18512 chars
1986-10-22 Ronald Wilson Reagan.txt = 8146 chars
1925-03-04 Calvin Coolidge.txt = 23742 chars
1936-06-27 Franklin Delano Roosevelt.txt = 11431 chars
1951-04-11 Harry S. Truman.txt = 13583 chars
1964-04-20 Lyndon Baines Johnson.txt = 32761 chars
1837-09-04 Martin Van Buren.txt = 32761 chars
1929-07-24 Herbert Clark Hoover.txt = 6579 chars
1939-09-03 Franklin Delano Roosevelt.txt = 6785 c

1981-01-29 Ronald Wilson Reagan.txt = 24969 chars
1963-07-26 John Fitzgerald Kennedy.txt = 17814 chars
1861-12-03 Abraham Lincoln.txt = 32761 chars
1964-02-01 Lyndon Baines Johnson.txt = 22373 chars
1982-09-01 Ronald Wilson Reagan.txt = 16966 chars
1901-03-04 William McKinley.txt = 13308 chars
1974-04-29 Richard Milhous Nixon.txt = 27282 chars
1806-01-10 Thomas Jefferson.txt = 4423 chars
1872-12-02 Ulysses S. Grant.txt = 32761 chars
1973-01-20 Richard Milhous Nixon.txt = 9811 chars
1834-12-01 Andrew Jackson.txt = 32761 chars
1863-12-08 Abraham Lincoln.txt = 32761 chars
1961-04-20 John Fitzgerald Kennedy.txt = 8640 chars
1966-10-06 Lyndon Baines Johnson.txt = 22268 chars
1881-03-04 James Abram Garfield.txt = 17623 chars
1926-12-07 Calvin Coolidge.txt = 32761 chars
1987-12-08 Ronald Wilson Reagan.txt = 5543 chars
2001-01-18 Bill Clinton.txt = 6304 chars
1940-07-19 Franklin Delano Roosevelt.txt = 19868 chars
1945-03-01 Franklin Delano Roosevelt.txt = 32761 chars
1981-11-18 Ronald Wilson R

1837-03-04 Martin Van Buren.txt = 23214 chars
1860-02-27 Abraham Lincoln.txt = 32761 chars
1799-12-03 John Adams.txt = 9054 chars
1964-01-08 Lyndon Baines Johnson.txt = 18315 chars
1984-08-23 Ronald Wilson Reagan.txt = 28980 chars
1968-10-31 Lyndon Baines Johnson.txt = 11989 chars
1965-03-26 Lyndon Baines Johnson.txt = 4318 chars
1960-10-13 John Fitzgerald Kennedy.txt = 32761 chars
1885-12-08 Grover Cleveland.txt = 32761 chars
1792-11-06 George Washington.txt = 14027 chars
1967-09-29 Lyndon Baines Johnson.txt = 16015 chars
1980-08-14 Jimmy Carter.txt = 26537 chars
1960-09-26 John Fitzgerald Kennedy.txt = 32761 chars
1893-03-04 Grover Cleveland.txt = 12277 chars
1938-07-08 Franklin Delano Roosevelt.txt = 8288 chars
2003-01-28 George W. Bush.txt = 31452 chars
1983-03-23 Ronald Wilson Reagan.txt = 26374 chars
1993-01-20 Bill Clinton.txt = 8961 chars
1963-11-28 Lyndon Baines Johnson.txt = 11003 chars
1971-01-22 Richard Milhous Nixon.txt = 25407 chars
1943-09-08 Franklin Delano Roosevelt.tx

In [6]:
%%time

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])
print('\n')

[[u'good', u'evening', u'my', u'fellow', u'americans', u'ten', u'days', u'ago', u'in', u'my', u'report', u'to', u'the', u'nation', u'on', u'vietnam', u'announced', u'decision', u'to', u'withdraw', u'an', u'additional', u'americans', u'from', u'vietnam', u'over', u'the', u'next', u'year', u'said', u'then', u'that', u'was', u'making', u'that', u'decision', u'despite', u'our', u'concern', u'over', u'increased', u'enemy', u'activity', u'in', u'laos', u'in', u'cambodia', u'and', u'in', u'south', u'vietnam', u'at', u'that', u'time', u'warned', u'that', u'if', u'concluded', u'that', u'increased', u'enemy', u'activity', u'in', u'any', u'of', u'these', u'areas', u'endangered', u'the', u'fives', u'of', u'americans', u'remaining', u'in', u'vietnam', u'would', u'not', u'hesitate', u'to', u'take', u'strong', u'and', u'effective', u'measures', u'to', u'deal', u'with', u'that', u'situation', u'despite', u'that', u'warning', u'north', u'vietnam', u'has', u'increased', u'its', u'military', u'aggression

In [7]:
%%time

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])
print('\n')

[u'good_evening', u'my', u'fellow_americans', u'ten', u'days', u'ago', u'in', u'my', u'report', u'to', u'the', u'nation', u'on', u'vietnam', u'announced', u'decision', u'to', u'withdraw', u'an', u'additional', u'americans', u'from', u'vietnam', u'over', u'the', u'next', u'year', u'said', u'then', u'that', u'was', u'making', u'that', u'decision', u'despite', u'our', u'concern', u'over', u'increased', u'enemy', u'activity', u'in', u'laos', u'in', u'cambodia', u'and', u'in', u'south_vietnam', u'at', u'that', u'time', u'warned', u'that', u'if', u'concluded', u'that', u'increased', u'enemy', u'activity', u'in', u'any', u'of', u'these', u'areas', u'endangered', u'the', u'fives', u'of', u'americans', u'remaining', u'in', u'vietnam', u'would', u'not', u'hesitate', u'to', u'take', u'strong', u'and', u'effective', u'measures', u'to', u'deal', u'with', u'that', u'situation', u'despite', u'that', u'warning', u'north_vietnam', u'has', u'increased', u'its', u'military', u'aggression', u'in', u'all',

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
%%time
# This cell takes 2-3 minutes to run on my machine.  -j

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])
print('\n')

[[u'good_even', u'fellow', u'american', u'day', u'ago', u'report', u'nation', u'vietnam', u'announce', u'decision', u'withdraw', u'additional', u'american', u'vietnam', u'next', u'year', u'say', u'make', u'decision', u'concern', u'increase', u'enemy', u'activity', u'laos', u'cambodia', u'south_vietnam', u'time', u'warn', u'conclude', u'increase', u'enemy', u'activity', u'area', u'endanger', u'five', u'american', u'remain', u'vietnam', u'hesitate', u'take', u'strong', u'effective', u'measure', u'deal', u'situation', u'warn', u'north_vietnam', u'increase', u'military', u'aggression', u'area', u'particularly', u'cambodia', u'full', u'consultation', u'national', u'security_council', u'ambassador_bunker', u'general_abram', u'adviser', u'conclude', u'action', u'enemy', u'last', u'day', u'clearly', u'endanger', u'life', u'american', u'vietnam', u'constitute', u'unacceptable', u'risk', u'withdrawal', u'protect', u'man', u'vietnam', u'guarantee', u'continued', u'success', u'withdrawal', u'vietn

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 16), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 3), (14, 3), (15, 3), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 2), (25, 25), (26, 1), (27, 1), (28, 2), (29, 4), (30, 2), (31, 1), (32, 1), (33, 9), (34, 1), (35, 2), (36, 2), (37, 4), (38, 1), (39, 5), (40, 1), (41, 12), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 3), (52, 1), (53, 4), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 3), (65, 2), (66, 1), (67, 1), (68, 2), (69, 20), (70, 6), (71, 1), (72, 1), (73, 3), (74, 2), (75, 1), (76, 2), (77, 2), (78, 2), (79, 1), (80, 1), (81, 2), (82, 2), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 6), (91, 1), (92, 2), (93, 4), (94, 2), (95, 2), (96, 1), (97, 1), (98, 1), (99, 2), (100, 5), (101, 1), (102, 1), (103, 1), (104, 4), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110

In [11]:
id2word[0]

u'abroad'

In [12]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(u'abroad', 1),
  (u'absolute', 1),
  (u'accept', 1),
  (u'accomplish', 1),
  (u'accordingly', 1),
  (u'act', 2),
  (u'action', 16),
  (u'active', 1),
  (u'activity', 2),
  (u'additional', 1),
  (u'administration', 1),
  (u'adviser', 1),
  (u'age', 1),
  (u'aggression', 3),
  (u'ago', 3),
  (u'agreement', 3),
  (u'aid', 1),
  (u'air', 2),
  (u'airstrip', 1),
  (u'allow', 1),
  (u'ally', 1),
  (u'already', 1),
  (u'also', 2),
  (u'ambassador_bunker', 1),
  (u'america', 2),
  (u'american', 25),
  (u'ammunition', 1),
  (u'amount', 1),
  (u'anarchy', 2),
  (u'announce', 4),
  (u'answer', 2),
  (u'approach', 1),
  (u'appropriate', 1),
  (u'area', 9),
  (u'arm', 1),
  (u'armed_force', 2),
  (u'army', 2),
  (u'ask', 4),
  (u'assail', 1),
  (u'assistance', 5),
  (u'assume', 1),
  (u'attack', 12),
  (u'attitude', 1),
  (u'august', 1),
  (u'avoid', 2),
  (u'aware', 1),
  (u'away', 1),
  (u'baltimore', 1),
  (u'band', 1),
  (u'base', 2),
  (u'battlefield', 1),
  (u'beak', 3),
  (u'because', 1),

In [13]:
%%time

### Much as we saw with Word2vec, these settings are the key to tuning your LDA Topic Model. ###

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

CPU times: user 23.9 s, sys: 28.2 ms, total: 23.9 s
Wall time: 12 s


In [14]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  u'0.018*"state" + 0.011*"government" + 0.009*"united" + 0.007*"country" + 0.006*"congress" + 0.006*"public" + 0.005*"make" + 0.005*"power" + 0.004*"interest" + 0.004*"year"'),
 (1,
  u'0.012*"year" + 0.012*"people" + 0.011*"president" + 0.011*"american" + 0.008*"work" + 0.008*"go" + 0.007*"say" + 0.007*"time" + 0.007*"think" + 0.006*"government"'),
 (2,
  u'0.009*"government" + 0.009*"business" + 0.007*"man" + 0.007*"law" + 0.007*"great" + 0.006*"country" + 0.006*"make" + 0.006*"people" + 0.005*"power" + 0.005*"court"'),
 (3,
  u'0.014*"nation" + 0.013*"world" + 0.010*"peace" + 0.010*"people" + 0.008*"war" + 0.007*"man" + 0.007*"american" + 0.006*"united" + 0.006*"great" + 0.006*"state"')]


In [15]:
%%time

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print('\n')

('\nPerplexity: ', -7.792669462002289)
('\nCoherence Score: ', 0.3327504347867296)


CPU times: user 7.18 s, sys: 210 ms, total: 7.39 s
Wall time: 32.3 s


In [16]:
%%time

# Visualize the topics
#
# If you get an error like this: "pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. 
# A future version of pandas will change to not sort by default."
#
# then from the command line do: "pip install pandas==0.21.0"

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)  # sort=False ? sort=True
vis
print('\n')

/anaconda2/lib/python2.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))




CPU times: user 2.71 s, sys: 186 ms, total: 2.9 s
Wall time: 8.86 s


#import pandas as pd
#pd.__version__

vis

In [17]:
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      31.871456        1       1  0.135391  0.068865
1      28.425888        1       2 -0.102352 -0.072734
3      27.222162        1       3 -0.113831  0.087763
2      12.480499        1       4  0.080792 -0.083894, topic_info=      Category         Freq          Term        Total  loglift  logprob
term                                                                    
612    Default  4141.000000         world  4141.000000  30.0000  30.0000
14580  Default  3349.000000     president  3349.000000  29.0000  29.0000
19447  Default  1443.000000      business  1443.000000  28.0000  28.0000
14931  Default  3074.000000           man  3074.000000  27.0000  27.0000
5248   Default  3431.000000         peace  3431.000000  26.0000  26.0000
18890  Default  2276.000000           law  2276.000000  25.0000  25.0000
12001  Default  5336.000000        nation  5336.000000  24.0000  24.0000
12906  Default  6376.000000    government  6376.000000  23.0000  23.0000
6372   Default  3563.000000      congress  3563.000000  22.0000  22.0000
8266   Default  2481.000000            go  2481.000000  21.0000  21.0000
13314  Default  3140.000000           war  3140.000000  20.0000  20.0000
16759  Default   670.000000         court   670.000000  19.0000  19.0000
15242  Default  2541.000000        public  2541.000000  18.0000  18.0000
3325   Default  1378.000000           get  1378.000000  17.0000  17.0000
20101  Default  8149.000000         state  8149.000000  16.0000  16.0000
2289   Default  1113.000000       justice  1113.000000  15.0000  15.0000
5503   Default   898.000000           job   898.000000  14.0000  14.0000
13830  Default  1860.000000          help  1860.000000  13.0000  13.0000
18042  Default  2694.000000         power  2694.000000  12.0000  12.0000
14439  Default  1762.000000       freedom  1762.000000  11.0000  11.0000
18862  Default  3048.000000          work  3048.000000  10.0000  10.0000
14980  Default  2268.000000         think  2268.000000   9.0000   9.0000
60     Default  1898.000000          want  1898.000000   8.0000   8.0000
10496  Default  1168.000000  constitution  1168.000000   7.0000   7.0000
10276  Default  1382.000000        treaty  1382.000000   6.0000   6.0000
18487  Default  3854.000000         great  3854.000000   5.0000   5.0000
4611   Default  1550.000000         today  1550.000000   4.0000   4.0000
2234   Default  5246.000000      american  5246.000000   3.0000   3.0000
1206   Default  1157.000000       tonight  1157.000000   2.0000   2.0000
16535  Default  1174.000000       federal  1174.000000   1.0000   1.0000
...        ...          ...           ...          ...      ...      ...
15510   Topic4   154.478943          wage   266.481812   1.5358  -6.5721
10279   Topic4   193.888245    enterprise   366.831818   1.4434  -6.3449
14931   Topic4   819.664795           man  3074.570557   0.7590  -4.9033
9942    Topic4   546.521057      national  1965.172852   0.8012  -5.3086
10576   Topic4   112.383560        profit   178.254807   1.6197  -6.8902
8791    Topic4   336.378815        matter   967.991455   1.0240  -5.7939
10496   Topic4   376.824188  constitution  1168.551636   0.9493  -5.6804
19948   Topic4   313.308044       control   919.097107   1.0048  -5.8650
6397    Topic4   343.324799     condition  1079.051147   0.9358  -5.7735
3025    Topic4   233.418335    commission   577.295898   1.1755  -6.1593
12906   Topic4  1004.458923    government  6376.417480   0.2328  -4.7000
16535   Topic4   358.293732       federal  1174.559082   0.8937  -5.7308
18487   Topic4   726.865540         great  3854.491943   0.4127  -5.0234
18042   Topic4   585.430725         power  2694.804199   0.5543  -5.2398
3235    Topic4   414.331940        policy  1591.169556   0.7354  -5.5855
15494   Topic4   382.960266       purpose  1464.275146   0.7398  -5.6642
14760   Topic4   630.947327          make  3859.964111  